# Porg Decrypt

Hello,

At great cost our undercover agents have infiltrated the systems of the evil porg-napping ring. They managed to exfiltrate 2 files.  

We believe these files are tar files which have been xor encypted with a repeating three byte key.

Our agents also found a post it note with `c0fefe` written on it, we think it is a key to one of the files.  

We have no information on what the other key might be, though we suspect that the filenames are the md5sum of the 3 byte binary key.

Please write a program that can discover the key and decrypt both of these files so we can rescue porg!


### Define helper functions and imports

In [42]:
from itertools import cycle
from hashlib import md5
from os.path import basename

In [47]:
def key_to_bytes(key):
    if isinstance(key, tuple) or isinstance(key, list):
        return bytes([int(b) for b in key])
    if isinstance(key, bytes):
        return key
    return bytes([int(key[i:i+2], 16) for i in range(0, len(key), 2)])

In [48]:
def md5_match(match_str, key):
    return md5(key_to_bytes(key)).hexdigest() == match_str

In [60]:
# validate helper functions
assert key_to_bytes('c0fefe') == b'\xc0\xfe\xfe'
assert key_to_bytes(('192','254','254')) == b'\xc0\xfe\xfe'

assert md5_match('fde519a6ed032c4cd4e9fbab9be61890', 'c0fefe')
assert md5_match('fde519a6ed032c4cd4e9fbab9be61890', (192, 254, 254))

assert not md5_match('fde519a6ed032c4cd4e9fbab9be61890', 'c0feff')
assert not md5_match('fde519a6ed032c4cd4e9fbab9be61890', (192, 254, 255))

In [61]:
# check to see if our known key matches a filename
from os import listdir
known_key = 'c0fefe'
for file_name in listdir('.'):
    if md5_match(file_name.split('.')[0], known_key):
        print('Found match for file:' + file_name)
        break

Found match for file:fde519a6ed032c4cd4e9fbab9be61890.tar.enc


### Take a look at the first file with a known key

In [50]:
def peek(file_nm, key, num_bytes=256):
    key = key_to_bytes(key)
    with open(file_nm, 'rb') as in_file:
        raw_data = in_file.read(num_bytes)
        return ''.join(map(lambda raw_byte, key_byte: chr(raw_byte ^ key_byte), raw_data, cycle(key)))

In [51]:
peek('fde519a6ed032c4cd4e9fbab9be61890.tar.enc', 'c0fefe')

'ransom.pdf\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x000000744\x00\x80\x00\x00\x00r\x0bÓ\t\x80\x00\x00\x00r\x0bÓ\t00000522563\x0013337323762\x00012461\x00 0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

### Look at all of those zeros!

We know that `A XOR 0 = A`

So any place in the unencrypted file that contains repeating zeros will contain the key repeated over and over in the encrypted file.

We can use that to help find the key without having to brute force it

In [52]:
def find_key(file_nm, key_len=3):
    # remove file extensions from file name to use as md5 to match
    match_str = basename(file_nm).split('.')[0]
    num_bytes = 512
    with open(file_nm, 'rb') as input_file:
        # read bytes from inpute file
        raw_bytes = input_file.read(num_bytes)
        for i in range(num_bytes - key_len):
            # check to see if these bytes contain the key we need
            key = raw_bytes[i:i+key_len]
            if md5_match(match_str, key):
                return key
    return None

In [62]:
# make sure that we get 'c0fefe' out of our first file
assert find_key('fde519a6ed032c4cd4e9fbab9be61890.tar.enc') == b'\xc0\xfe\xfe'

### Once we have the key, we can decrypt the file!

In [68]:
def decrypt(file_name, key, out_path):
    with open(file_name, 'rb') as input_file, open(out_path, 'wb') as output_file:
        output_file.write(bytes([b for b in map(lambda raw_byte, key_byte: raw_byte ^ key_byte, input_file.read(), cycle(key_to_bytes(key)))]))

        

In [69]:
decrypt('fde519a6ed032c4cd4e9fbab9be61890.tar.enc', 'c0fefe', 'fde519a6ed032c4cd4e9fbab9be61890.tar')

In [99]:
!ls

389eb0c70f3a36eeda39510afcb2ea08.tar.enc  porg_decrypt.ipynb
389.tar					  ransom.pdf
fde519a6ed032c4cd4e9fbab9be61890.tar	  thanks.pdf
fde519a6ed032c4cd4e9fbab9be61890.tar.enc  Untitled.ipynb
output


In [74]:
!tar -xvf fde519a6ed032c4cd4e9fbab9be61890.tar

ransom.pdf


In [104]:
from IPython.display import IFrame
IFrame('ransom.pdf', width=800, height=1100)

### Put it all together and we can decrypt the other file

In [96]:
key = find_key('389eb0c70f3a36eeda39510afcb2ea08.tar.enc')
print('found key: ' + str(key))
decrypt('389eb0c70f3a36eeda39510afcb2ea08.tar.enc', key, '389.tar')

found key: b'\x0b\xee\xf0'


In [97]:
!tar -xvf 389.tar

thanks.pdf


In [106]:
from IPython.display import IFrame
IFrame('thanks.pdf', width=800, height=1000)